## Writing a Simple Action Server using the Execute Callback
This tutorial covers using the simple_action_server library to create a Fibonacci action server in Python. This example action server generates a Fibonacci sequence, the goal is the order of the sequence, the feedback is the sequence as it is computed, and the result is the final sequence.

In [ ]:
import rospy
import actionlib
import actionlib_tutorials.msg

In [ ]:
action_name = 'fibonacci'
feedback = actionlib_tutorials.msg.FibonacciFeedback()
result = actionlib_tutorials.msg.FibonacciResult()

In [ ]:
def execute_cb(goal):
    r = rospy.Rate(1)
    success = True
    feedback.sequence = []
    feedback.sequence.append(0)
    feedback.sequence.append(1)
    rospy.loginfo('%s: Executing, creating fibonacci sequence of order %i with seeds %i, %i' % 
                  (action_name, goal.order, feedback.sequence[0], feedback.sequence[1]))
    for i in range(1, goal.order):
            if action_server.is_preempt_requested():
                rospy.loginfo('%s: Preempted' % self._action_name)
                action_server.set_preempted()
                success = False
                break
            feedback.sequence.append(feedback.sequence[i] + feedback.sequence[i-1])
            action_server.publish_feedback(feedback)
            r.sleep()          
    if success:
            result.sequence = feedback.sequence
            rospy.loginfo('%s: Succeeded' % action_name)
            action_server.set_succeeded(result)

In [ ]:
action_server = actionlib.SimpleActionServer(action_name, actionlib_tutorials.msg.FibonacciAction, 
                                  execute_cb = execute_cb, auto_start = False)

In [ ]:
rospy.init_node('fibonacci')
action_server.start()
rospy.spin()

[Writing a Simple Action Client](Client.ipynb).